In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy, scipy as sklearn, librosa, urllib
import librosa.display
from IPython.display import Audio
import json 
import seaborn as sns
from sklearn.cluster import KMeans
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import keras
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from itertools import cycle
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, silhouette_score,roc_auc_score, precision_recall_fscore_support

In [2]:
#read the pickle files
not_aug_df=pd.read_pickle('EMOVO_dataset/not_aug_df.pkl')
semi_aug_df=pd.read_pickle('EMOVO_dataset/semi_aug_df.pkl')
aug_df=pd.read_pickle('EMOVO_dataset/aug_df.pkl')

scaler = StandardScaler()

In [3]:
test_df = not_aug_df.loc[not_aug_df["actor"] == "f1"]
not_aug_df = not_aug_df.drop(test_df.index)

Y_test = test_df["label_id"]
X_mfccs_k_test = np.array(test_df.iloc[:, 3:29])
X_mfccs_k_test=np.array(X_mfccs_k_test.tolist())
X_mfccs_k_test=scaler.fit_transform(X_mfccs_k_test.reshape(-1, X_mfccs_k_test.shape[-1])).reshape(X_mfccs_k_test.shape)
Y_mfccs_k_test=test_df['label']
X_logmel_test = np.array(test_df.loc[:, ['logmel' in i for i in test_df.columns]])
X_logmel_test=np.array(X_logmel_test.tolist())
X_test = np.reshape(X_logmel_test, (X_logmel_test.shape[0],X_logmel_test.shape[1],X_logmel_test.shape[2],1))
X_logmel_test = scaler.fit_transform(X_logmel_test.reshape(-1, X_logmel_test.shape[-1])).reshape(X_logmel_test.shape)


In [4]:
X_test.shape

(98, 60, 130, 1)

In [5]:
X_logmel_test.shape

(98, 60, 130)

In [6]:
len(not_aug_df)

490

In [4]:
#creation of the array containg the emotions ordered by their encoding
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
sorted_labels=[]
label_encoder.fit(not_aug_df['label'])
name_mapping = dict(zip( label_encoder.transform(label_encoder.classes_),label_encoder.classes_))
for i in range(7):
  sorted_labels.append(name_mapping[i])
sorted_labels

['anger', 'disgust', 'fear', 'joy', 'neutrality', 'sadness', 'surprise']

In [5]:
scaler = StandardScaler()
#extraction of the mfccs from the datasets - not aug
X_mfccs_k = np.array(not_aug_df.iloc[:, 3:29])
X_mfccs_k=np.array(X_mfccs_k.tolist())
X_mfccs_k=scaler.fit_transform(X_mfccs_k.reshape(-1, X_mfccs_k.shape[-1])).reshape(X_mfccs_k.shape)
Y_mfccs_k=not_aug_df['label']


#extraction of the log mel specrogram from the datasets - not aug
X_logmel_k = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
X_logmel_k=np.array(X_logmel_k.tolist())
X_logmel_k=scaler.fit_transform(X_logmel_k.reshape(-1, X_logmel_k.shape[-1])).reshape(X_logmel_k.shape)
Y_logmel_k=not_aug_df['label']

#reshape the data from 3D to 2D - not aug
X_mfccs_k=X_mfccs_k.reshape(X_mfccs_k.shape[0],X_mfccs_k.shape[1]*X_mfccs_k.shape[2])
X_logmel_k=X_logmel_k.reshape(X_logmel_k.shape[0],X_logmel_k.shape[1]*X_logmel_k.shape[2])

In [6]:
X_logmel_k = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])


In [7]:
#extraction of labels_id from datasets
Y_not_aug=not_aug_df['label_id']
Y_semi_aug=semi_aug_df['label_id']
Y_aug=aug_df['label_id']


#take the log mel spectrogram from the datasets
X_logmel = np.array(not_aug_df.loc[:, ['logmel' in i for i in not_aug_df.columns]])
X_logmel=np.array(X_logmel.tolist())
X_logmel_semi_aug = np.array(semi_aug_df.loc[:, ['logmel' in i for i in semi_aug_df.columns]])
X_logmel_semi_aug=np.array(X_logmel_semi_aug.tolist())
X_logmel_aug = np.array(aug_df.loc[:, ['logmel' in i for i in aug_df.columns]])
X_logmel_aug=np.array(X_logmel_aug.tolist())


#take the mfccs & deltas from the datasets
X_mfccs = np.array(not_aug_df.iloc[:, 3:29]) 
X_mfccs=np.array(X_mfccs.tolist())
X_mfccs_semi_aug = np.array(semi_aug_df.iloc[:, 3:29])
X_mfccs_semi_aug=np.array(X_mfccs_semi_aug.tolist())
X_mfccs_aug = np.array(aug_df.iloc[:, 3:29])
X_mfccs_aug=np.array(X_mfccs_aug.tolist())

In [8]:
X_logmel.shape

(490, 60, 130)

In [9]:
#add one dimension to data to make them suitable foR CNN
X_logmel = np.reshape(X_logmel, (X_logmel.shape[0],X_logmel.shape[1],X_logmel.shape[2],1))
X_logmel_semi_aug= np.reshape(X_logmel_semi_aug, (X_logmel_semi_aug.shape[0],X_logmel_semi_aug.shape[1],X_logmel_semi_aug.shape[2],1))
X_logmel_aug= np.reshape(X_logmel_aug, (X_logmel_aug.shape[0],X_logmel_aug.shape[1],X_logmel_aug.shape[2],1))

In [13]:
X_logmel.shape

(490, 60, 130, 1)

In [17]:
def create_cnn(input_shape):
    model = keras.Sequential()

    model.add(keras.layers.Input(shape=input_shape))

    model.add(keras.layers.Conv2D(256, 3, activation='relu' ))
    model.add(keras.layers.MaxPooling2D(padding='same'))
    model.add(keras.layers.Dropout(rate=0.3))

    model.add(keras.layers.Conv2D(128, 3, activation='relu'))
    model.add(keras.layers.MaxPooling2D(padding='same'))
    model.add(keras.layers.Dropout(rate=0.3))


    model.add(keras.layers.Conv2D(64, 3, activation='relu'))
    model.add(keras.layers.MaxPooling2D(padding='same'))
    model.add(keras.layers.Dropout(rate=0.3))

    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dense(1024, activation='relu'))
    
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dense(64, activation='relu'))

    model.add(keras.layers.Dense(7, activation='softmax'))

    optimzer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy', optimizer=optimzer, metrics=['accuracy'])

    return model

In [11]:
X_logmel.shape

(490, 60, 130, 1)

In [38]:
def compile_model(input):
    input_data=(input.shape[1:])
    modelcnn=create_cnn(input_data)
    modelcnn.compile(
    optimizer = "Adam", 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
    )
    return modelcnn
def train_val_test(X,Y):
    X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
    Y = keras.utils.to_categorical(Y.to_numpy())
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=22)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=22)
    return X_train,X_val,X_test,y_train,y_val,y_test

def get_accuracy(X_test,y_test, modelcnn):
    predicted = modelcnn.predict(X_test)

    preds = []
    for prediction in predicted:
        preds.append(np.argmax(prediction))

    print(accuracy_score(y_test,preds))


In [28]:
from datetime import datetime  
name = datetime.now().strftime("ser_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=10,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]

X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel,Y_not_aug)
modelcnn = compile_model(X_train)

# modelcnn.summary()

history = modelcnn.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=256,
                       epochs=1000,
                       callbacks=callbacks
                       )

Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1360 - loss: 1.9465  
Epoch 1: val_loss improved from inf to 1.93669, saving model to ser_07_10_2024_16_31_12.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.1371 - loss: 1.9465 - val_accuracy: 0.1910 - val_loss: 1.9367
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2420 - loss: 1.9182
Epoch 2: val_loss improved from 1.93669 to 1.91037, saving model to ser_07_10_2024_16_31_12.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.2380 - loss: 1.9201 - val_accuracy: 0.2022 - val_loss: 1.9104
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2173 - loss: 1.8775
Epoch 3: val_loss improved from 1.91037 to 1.90137, saving model to ser_07_10_2024_16_31_12.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.2169 - loss: 1.8766 - val_accuracy: 0.1910 - val_loss: 1.9014
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2362 - loss: 1.8480
Epoch 4: val_loss improved from 1.90137

In [30]:
get_accuracy(X_test,y_test,modelcnn)
print(f"Loss : {modelcnn.evaluate(X_test,y_test)[0]}, Accuracy : {modelcnn.evaluate(X_test,y_test)[1]}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step 
0.46938775510204084
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4900 - loss: 1.3877
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4900 - loss: 1.3877
Loss : 1.4225701093673706, Accuracy : 0.4693877696990967


In [25]:


X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_semi_aug,Y_semi_aug)
modelcnn = compile_model(X_train    )

In [26]:
from datetime import datetime  
name = datetime.now().strftime("ser_semiaug_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.01,
        patience=10,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]

history_semi = modelcnn.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=256,
                       epochs=1000,
                       callbacks=callbacks)

Epoch 1/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1491 - loss: 1.9391
Epoch 1: val_loss improved from inf to 1.89750, saving model to ser_semiaug_07_10_2024_11_43_13.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.1525 - loss: 1.9376 - val_accuracy: 0.2516 - val_loss: 1.8975
Epoch 2/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2207 - loss: 1.8686
Epoch 2: val_loss improved from 1.89750 to 1.87293, saving model to ser_semiaug_07_10_2024_11_43_13.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.2206 - loss: 1.8683 - val_accuracy: 0.2516 - val_loss: 1.8729
Epoch 3/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2290 - loss: 1.8347
Epoch 3: val_loss improved from 1.87293 to 1.85809, saving model to ser_semiaug_07_10_2024_11_43_13.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.2301 - loss: 1.8354 - val_accuracy: 0.2296 - val_loss: 1.8581
Epoch 4/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2521 - loss: 1.8083
Epoch 4: val_lo

In [27]:
y_train_ = np.array(y_train)

In [28]:
get_accuracy(X_test,y_test,modelcnn)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step
0.8757062146892656


In [13]:
modelcnn = compile_model()

X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_aug,Y_aug)

In [14]:
from datetime import datetime  
name = datetime.now().strftime("ser_aug_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=10,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


history_semi = modelcnn.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=256,
                       epochs=1000,
                       callbacks=callbacks
                       )

Epoch 1/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1504 - loss: 1.9342
Epoch 1: val_loss improved from inf to 1.89338, saving model to ser_aug_07_10_2024_15_13_24.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.1531 - loss: 1.9324 - val_accuracy: 0.2311 - val_loss: 1.8934
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2167 - loss: 1.8876
Epoch 2: val_loss improved from 1.89338 to 1.85522, saving model to ser_aug_07_10_2024_15_13_24.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.2171 - loss: 1.8860 - val_accuracy: 0.2476 - val_loss: 1.8552
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2435 - loss: 1.8447
Epoch 3: val_loss improved from 1.85522 to 1.84853, saving model to ser_aug_07_10_2024_15_13_24.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.2422 - loss: 1.8452 - val_accuracy: 0.2406 - val_loss: 1.8485
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2518 - loss: 1.8319
Epoch 4: val_loss improved 

In [16]:
get_accuracy(X_test,y_test,modelcnn)

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step
0.885593220338983


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from keras import layers, models


In [13]:
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel,Y_not_aug)
X_train.shape

(423, 60, 130, 1)

In [39]:
def get_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    encoder = layers.LSTM(128)(inputs)
    drop = layers.Dropout(0.3)(encoder)
    hidden = layers.Dense(32, activation='relu')(drop)
    outputs = layers.Dense(7, activation='softmax')(hidden)
    
    model = models.Model(inputs, outputs)
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

    return model

In [40]:
(X_train.shape[1:])

(60, 130)

In [41]:
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel.squeeze(3),Y_not_aug)
LSTM_model = get_model(X_train.shape[1:])
LSTM_model.summary()



Model: "functional_71"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 60, 130)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       132,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,967 (535.03 KB)

 Trainable params: 136,967 (535.03 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:

from datetime import datetime  
name = datetime.now().strftime("ser_lstm_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


LSTM_history = LSTM_model.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=32,
                       epochs=1000,
                       verbose=1,
                       callbacks=callbacks)


print(f"Loss : {LSTM_model.evaluate(X_test,y_test)[0]}, Accuracy : {LSTM_model.evaluate(X_test,y_test)[1]}")

Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1205 - loss: 1.9909
Epoch 1: val_loss improved from inf to 1.97996, saving model to ser_lstm_07_10_2024_16_50_16.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 157ms/step - accuracy: 0.1230 - loss: 1.9899 - val_accuracy: 0.1461 - val_loss: 1.9800
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.2468 - loss: 1.8175
Epoch 2: val_loss improved from 1.97996 to 1.93957, saving model to ser_lstm_07_10_2024_16_50_16.keras
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.2471 - loss: 1.8185 - val_accuracy: 0.1798 - val_loss: 1.9396
Epoch 3/1000
10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.3694 - loss: 1.7400
Epoch 3: val_loss did not improve from 1.93957
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.3665 - loss: 1.7408 - val_accuracy: 0.1910 - val_loss: 1.9505
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4225 - loss: 1.6029
Epoch 4: val_loss did not improve from 1.93957
11/11 ━

In [45]:
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_semi_aug.squeeze(3),Y_semi_aug)
LSTM_model = get_model(X_train.shape[1:])
# LSTM_model.summary()


from datetime import datetime  
name = datetime.now().strftime("ser_lstm_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


LSTM_history = LSTM_model.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=256,
                       epochs=1000,
                       verbose=1,
                       callbacks=callbacks)


print(f"Loss : {LSTM_model.evaluate(X_test,y_test)[0]}, Accuracy : {LSTM_model.evaluate(X_test,y_test)[1]}")

Epoch 1/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step - accuracy: 0.1573 - loss: 2.0561
Epoch 1: val_loss improved from inf to 1.90702, saving model to ser_lstm_07_10_2024_16_59_56.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 701ms/step - accuracy: 0.1588 - loss: 2.0464 - val_accuracy: 0.2201 - val_loss: 1.9070
Epoch 2/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 0.2239 - loss: 1.8929
Epoch 2: val_loss improved from 1.90702 to 1.88487, saving model to ser_lstm_07_10_2024_16_59_56.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 390ms/step - accuracy: 0.2254 - loss: 1.8906 - val_accuracy: 0.2579 - val_loss: 1.8849
Epoch 3/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step - accuracy: 0.2731 - loss: 1.8417
Epoch 3: val_loss improved from 1.88487 to 1.85692, saving model to ser_lstm_07_10_2024_16_59_56.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 485ms/step - accuracy: 0.2724 - loss: 1.8414 - val_accuracy: 0.2610 - val_loss: 1.8569
Epoch 4/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step - accuracy: 0.2985 - loss: 1.8021
Epoch 

In [46]:
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_aug.squeeze(3),Y_aug)
LSTM_model = get_model(X_train.shape[1:])
# LSTM_model.summary()


from datetime import datetime  
name = datetime.now().strftime("ser_lstm_%d_%m_%Y_%H_%M_%S.keras")  

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = name,
        save_best_only=True,
        verbose=1,
        monitor="val_loss"),

    keras.callbacks.EarlyStopping(  
        monitor="val_loss",
        min_delta=0.001,
        patience=20,
        verbose=1,
        mode="auto",
        restore_best_weights=True
    )
]


LSTM_history = LSTM_model.fit(X_train, y_train, 
                       validation_data=(X_val,y_val), 
                       batch_size=256,
                       epochs=1000,
                       verbose=1,
                       callbacks=callbacks)


print(f"Loss : {LSTM_model.evaluate(X_test,y_test)[0]}, Accuracy : {LSTM_model.evaluate(X_test,y_test)[1]}")

Epoch 1/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.1576 - loss: 1.9808
Epoch 1: val_loss improved from inf to 1.88135, saving model to ser_lstm_07_10_2024_17_04_19.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 619ms/step - accuracy: 0.1587 - loss: 1.9784 - val_accuracy: 0.2406 - val_loss: 1.8814
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step - accuracy: 0.2422 - loss: 1.8577
Epoch 2: val_loss improved from 1.88135 to 1.81651, saving model to ser_lstm_07_10_2024_17_04_19.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 475ms/step - accuracy: 0.2430 - loss: 1.8574 - val_accuracy: 0.3113 - val_loss: 1.8165
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step - accuracy: 0.2844 - loss: 1.8015
Epoch 3: val_loss improved from 1.81651 to 1.74696, saving model to ser_lstm_07_10_2024_17_04_19.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 408ms/step - accuracy: 0.2861 - loss: 1.8007 - val_accuracy: 0.3443 - val_loss: 1.7470
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step - accuracy: 0.3289 - loss: 1.7188
Epoch

In [47]:
from sklearn.svm import SVC
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel.squeeze(3),Y_not_aug)
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)


SVC_model = SVC(kernel = 'rbf', gamma = 'auto', probability = True, verbose=True)

In [58]:
SVC_history = SVC_model.fit(X_train, np.argmax(y_train,axis=-1))
SVC_model.score(X_test,np.argmax(y_test,axis=-1))

[LibSVM]

0.32653061224489793

In [59]:
from sklearn.svm import SVC
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_semi_aug.squeeze(3),Y_semi_aug)
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)


SVC_model = SVC(kernel = 'rbf', gamma = 'auto', probability = True, verbose=True)


SVC_history = SVC_model.fit(X_train, np.argmax(y_train,axis=-1))
SVC_model.score(X_test,np.argmax(y_test,axis=-1))

[LibSVM]

0.6384180790960452

In [60]:
from sklearn.svm import SVC
X_train,X_val,X_test,y_train,y_val,y_test = train_val_test(X_logmel_aug.squeeze(3),Y_aug)
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)


SVC_model = SVC(kernel = 'rbf', gamma = 'auto', probability = True, verbose=True)

SVC_history = SVC_model.fit(X_train, np.argmax(y_train,axis=-1))
SVC_model.score(X_test,np.argmax(y_test,axis=-1))

[LibSVM]

0.7966101694915254